In [ ]:
import torch
import pickle
import numpy as np
from tqdm import tqdm

In [ ]:
data="sensors"
fold=4
emb_path = f"{data}_fold_{fold}_pen_embs.pkl"

In [ ]:
with open(emb_path, 'rb') as f:
    embs = pickle.load(f)

In [ ]:
groups = np.array([v.item() for v in embs['groups']])
mean_embeddings = []
for group in tqdm(range(4)):
    group_indices = np.where(groups == group)
    group_embeddings = np.array(embs['embeddings'])[group_indices[0]]
    mean_embedding = np.mean(group_embeddings, axis=0)
    mean_embeddings.append(mean_embedding)
    
mean_embeddings_tensor = torch.tensor(mean_embeddings)
torch.save(mean_embeddings_tensor, f'{data}_mean_embeddings.pt')
mean_embeddings_tensor.shape